In [18]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import from_unixtime, when, col, lit

In [19]:
sc = SparkContext('local')
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at <ipython-input-5-3afc111ff308>:5 

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [11]:
df = spark.read.load('Files/class_data.csv',format="csv", sep=",", inferSchema="true", header="true")

In [15]:
total_events = df.count()
print('There are',total_events,'events')

There are 442 events


In [12]:
df.printSchema()

root
 |-- entry: integer (nullable = true)
 |-- is1L2Tau: boolean (nullable = true)
 |-- is2L2Tau: boolean (nullable = true)
 |-- is2LSS0Tau: boolean (nullable = true)
 |-- pass_tight_2L: boolean (nullable = true)
 |-- pass_SS_2L: boolean (nullable = true)
 |-- is2LSS1Tau: boolean (nullable = true)
 |-- is2LOS1Tau: boolean (nullable = true)
 |-- is3L: boolean (nullable = true)
 |-- is3L1Tau: boolean (nullable = true)
 |-- is4L: boolean (nullable = true)
 |-- is2lLowNJet0Tau: boolean (nullable = true)
 |-- is2LSS0Tau_SR_Relaxed: boolean (nullable = true)
 |-- is2LSS1Tau_SR_Relaxed: boolean (nullable = true)
 |-- is3L0Tau_SR_Relaxed: boolean (nullable = true)
 |-- is3L0Tau_SR_Relaxed3j: boolean (nullable = true)
 |-- is4L_SR_VV_Relaxed: boolean (nullable = true)
 |-- is4L_SR_ttV_Relaxed: boolean (nullable = true)
 |-- is4L_SR_ttV_Zenriched: boolean (nullable = true)
 |-- is3L0Tau_CR_ttbar_Relaxed: boolean (nullable = true)
 |-- is3L0Tau_CR_VV_Relaxed: boolean (nullable = true)
 |-- is3L0

In [13]:
df.groupBy('lep_flavour').count().show()

+-----------+-----+
|lep_flavour|count|
+-----------+-----+
|         -2|  442|
+-----------+-----+



In [20]:
df_data = df.withColumn("label",lit(1))

In [14]:
def compute_hist(data, feature, target='label', n_bins=100, x_lim=[0,3]):
        
    ## Fix the range
    data = data.where((col(feature)<=x_lim[1]) &
                      (col(feature)>=x_lim[0]))
    
    sgn = data.where(col(target)==1.0) 
    bkg = data.where(col(target)==0.0)

    ## Compute the histograms
    bins_sgn, counts_sgn = sgn.select(feature).rdd.flatMap(lambda x: x).histogram(n_bins)
    bins_bkg, counts_bkg = bkg.select(feature).rdd.flatMap(lambda x: x).histogram(n_bins)
    
    return (bins_sgn, counts_sgn), (bins_bkg, counts_bkg)

In [ ]:
%matplotlib notebook

## hist is a touple containing bins and counts foreach bin
hist_signal, hist_bkg = compute_hist(data=train, feature='Mll01', target='label', n_bins=50, x_lim=[0,3])

f, ax = plt.subplots()
ax.hist(hist_signal[0][:-1], bins=hist_signal[0], weights=hist_signal[1], alpha=0.5, label='signal')
ax.hist(hist_bkg[0][:-1], bins=hist_bkg[0], weights=hist_bkg[1], alpha=0.5, label='background')
ax.set_xlabel('$Mll01$')
ax.set_ylabel('counts')
ax.set_title("Distribution of $M_{l0l1}$")
ax.legend()
plt.show()